# groupby 메서드로 평균 값 구하기


In [4]:
import pandas as pd
df = pd.read_csv('../data/gapminder.tsv', sep = '\t')
df.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [3]:
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
#year 컬럼으로 그룹화 하여, lifeExp를 평균내라
avg_life_exp_by_year

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

# 분할 - 반영 - 결합 과정 살펴보기

In [5]:
#year 열의 데이터를 중복없이 추출, groupby 메서드에 열이름을 전달 -'분할'

years = df.year.unique()
#year 열의 데이터를 중복없이 추출 -> unique()
years

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007], dtype=int64)

In [7]:
#연도별로 평균값을 구하기 - '반영'

y1952 = df.loc[df.year == 1952, :]
#년도가 1952인 행들을 추출
y1952.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
12,Albania,Europe,1952,55.230,1282697,1601.056136
24,Algeria,Africa,1952,43.077,9279525,2449.008185
36,Angola,Africa,1952,30.015,4232095,3520.610273
48,Argentina,Americas,1952,62.485,17876956,5911.315053


In [8]:
y1952_mean = y1952.lifeExp.mean()
y1952_mean

49.05761971830987

In [13]:
# 위의 과정을 반복하여 남은 연도의 평균값을 구하면 '반영'작업이 끝남
y1957 = df.loc[df.year == 1957, :]
y1957_mean = y1957.lifeExp.mean()

y1962 = df.loc[df.year == 1962, :]
y1962_mean = y1962.lifeExp.mean()

In [15]:
df2 = pd.DataFrame({'year':[1952,1957,1962], "":[y1952_mean, y1957_mean, y1962_mean]})
# 분할 -반영 -> 데이터 프레임으로 결합하기
df2

,year,
0,1952,49.057620
1,1957,51.507401
2,1962,53.609249


# 평균값을 구하는 사용자 함수와 groupby 메서드
* 사용자 함수를 groupby 와 같이 사용하고자 할 때는 agg 메서드를 활용한다. 


In [17]:
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum +=value
    return sum/n

In [32]:
#사용자 함수를 groupby 와 같이 사용하고자 할 때는 agg 메서드를 활용한다. 
agg_mymean = df.groupby('year').lifeExp.agg(my_mean)
agg_mymean

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

## 두개의 인자값을 받아 처리하는 사용자 함수 agg() 
* agg 메서드의 첫번째 인자는, agg전의 호출 컬럼(ex- lifeExp)이고, 다른 인자는 인자명 = 넘겨줄 인자 식으로 따로 명시해주어야 한다. 

In [50]:
# 연도별 lifeExp 평균 - 전체 ligeExp 평균 
def my_mean_diff(val1, exp):
    return my_mean(val1)-exp

exp = df.lifeExp.agg(my_mean)


agg_mymean2 = df.groupby('year').lifeExp.agg(my_mean_diff, exp)
agg_mymean2


year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64

# 집계 메서드를 리스트, 딕셔너리에 담아 전달하기

In [55]:
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
# numpy 패키지를 통해 널값이 아닌 데이터값 갯수 추출 -> count_nonzero, 평균값 -> mean, 표준편차 -> std
gdf

,count_nonzero,mean,std
year,,,
1952,142.0,49.057620,12.225956
1957,142.0,51.507401,12.231286
1962,142.0,53.609249,12.097245
1967,142.0,55.678290,11.718858
1972,142.0,57.647386,11.381953
1977,142.0,59.570157,11.227229
1982,142.0,61.533197,10.770618
1987,142.0,63.212613,10.556285
1992,142.0,64.160338,11.227380


In [53]:
gdf_dict = df.groupby('year').agg({'lifeExp':'mean', 'pop': 'median', 'gdpPercap':'median'})
# 딕셔너리 형태로, year 별로 그룹화 하였을때의 칼럼값을 원하는 연산자('mean'. 'median' etc..)등으로 딕셔너리 형태로 묶어서 출력가능하다. 
gdf_dict

,lifeExp,pop,gdpPercap
year,,,
1952,49.057620,3943953.0,1968.528344
1957,51.507401,4282942.0,2173.220291
1962,53.609249,4686039.5,2335.439533
1967,55.678290,5170175.5,2678.334741
1972,57.647386,5877996.5,3339.129407
1977,59.570157,6404036.5,3798.609244
1982,61.533197,7007320.0,4216.228428
1987,63.212613,7774861.5,4280.300366
1992,64.160338,8688686.5,4386.085502


# 표준점수 계산하기

In [56]:
# 데이터의 평균과 표준편차의 차이를 표준점수라 한다.
def my_zscore(x):
    return (x - x.mean())/x.std()

### 데이터 변환

In [62]:
#데이터를 표준화 할 뿐 집계는 하지 않는다. 데이터 양이 줄어 들지 않음
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
transform_z.head()

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64

In [64]:
print(df.shape)
print(transform_z.shape)
# 양이 줄어들지 않는다. transform 메써드 

(1704, 6)
(1704,)


# 누락값을 평균값으로 처리하기

In [75]:
import seaborn as sns
import numpy as np
np.random.seed(42)
#난수 생성 고정

tips_10 = sns.load_dataset('tips').sample(10)
#sample(10) -> 10개(행)의 데이터셋을 추출

tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
#tips_10의 인덱스중, 랜덤으로 [:4], 4개까지 total_bill 에 np.NaN, NaN 값을 넣겠다 라는 뜻
tips_10

,total_bill,tip,sex,smoker,day,time,size
176,NaN,3.18,Male,No,Sat,Dinner,2
211,NaN,2.00,Male,No,Sun,Dinner,2
198,NaN,2.00,Male,No,Sun,Dinner,4
153,NaN,5.16,Male,Yes,Sat,Dinner,4
101,13.00,2.00,Female,Yes,Thur,Lunch,2
6,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
24,14.78,3.23,Male,No,Sun,Dinner,2
9,15.38,3.00,Female,Yes,Fri,Dinner,2


In [76]:
count_sex = tips_10.groupby('sex').count()
count_sex

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,3,7,7,7,7,7
Female,3,3,3,3,3,3


In [80]:
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [81]:
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
176         NaN  3.18    Male     No   Sat  Dinner     2            20.37
211         NaN  2.00    Male     No   Sun  Dinner     2            20.37
198         NaN  2.00    Male     No   Sun  Dinner     4            20.37
153         NaN  5.16    Male    Yes   Sat  Dinner     4            20.37
101       13.00  2.00  Female    Yes  Thur   Lunch     2            13.00
6         17.89  2.00    Male    Yes   Sun  Dinner     2            17.89
192       28.44  2.56    Male    Yes  Thur   Lunch     2            28.44
124       12.48  2.52  Female     No  Thur   Lunch     2            12.48
24        14.78  3.23    Male     No   Sun  Dinner     2            14.78
9         15.38  3.00  Female    Yes   Fri  Dinner     2            15.38


# 데이터 필터링 사용하기 -filter 메서드

In [82]:
tips = sns.load_dataset('tips')
print(tips.shape)

(244, 7)


In [83]:
tips['size'].value_counts()
#사이즈별 밸류 카운트하기 

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64

In [93]:
tips_filtered = tips.groupby('size').filter(lambda x : x['size'].count() >= 30)
#사이즈별 그룹화 하여 벨류 카운트 값이 30이상인 행만 필터링 하기
print(tips_filtered['size'].value_counts())
tips_filtered.head()

2    156
3     38
4     37
Name: size, dtype: int64


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# 그룹 오브젝트 저장하여 살펴보기

In [94]:
tips_10 = sns.load_dataset('tips').sample(10, random_state = 42)
#random_state 는 시드를 42로 고정하겠다는 뜻과같다. 
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,24.55,2.00,Male,No,Sun,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
198,13.00,2.00,Female,Yes,Thur,Lunch,2
176,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [97]:
# 자료형이 그룹 오브젝트 임을 확인
grouped = tips_10.groupby('sex')
grouped

In [98]:
# 그룹 오브젝트에 포함된 그룹을 볼려면 groups 속성
grouped.groups

{'Male': Int64Index([24, 6, 153, 211, 176, 192, 9], dtype='int64'),
 'Female': Int64Index([198, 124, 101], dtype='int64')}

In [99]:
avgs = grouped.mean()
#grouped = tips_10.groupby('sex')

avgs
#데이터중, 숫자타입의 컬럼에 한해서 mean() 이수행된다. 
#계산될 수 없는 열은 제외됨

,total_bill,tip,size
sex,,,
Male,20.02,2.875714,2.571429
Female,13.62,2.506667,2.000000


## 여러 컬럼을 그룹화 하고, 저장한뒤 산출하기 

In [108]:
bill_sex_time = tips_10.groupby(['sex', 'time'])
#sex 별로(female, male),그룹화한뒤, 각 그룹에서 다시 time(lunch. dinner)로 그룹화한다.

In [107]:
bill_sex_time.mean()

total_bill     tip
sex    time   size                    
Male   Lunch  2         28.440  2.5600
              4            NaN     NaN
       Dinner 2         15.315  2.6025
              4         25.220  3.5800
Female Lunch  2         12.740  2.2600
              4            NaN     NaN
       Dinner 2         15.380  3.0000
              4            NaN     NaN

In [109]:
group_method = tips_10.groupby(['sex', 'time'], as_index = False). mean()
group_method
#그룹화하기 위한 컬럼들을 index로 사용하고  싶지 않을때 ->1

,sex,time,total_bill,tip,size
0,Male,Lunch,28.440000,2.560000,2.000000
1,Male,Dinner,18.616667,2.928333,2.666667
2,Female,Lunch,12.740000,2.260000,2.000000
3,Female,Dinner,15.380000,3.000000,2.000000


In [110]:
group_method = tips_10.groupby(['sex', 'time']). mean().reset_index()
group_method
#그룹화하기 위한 컬럼들을 index로 사용하고  싶지 않을때 ->2

,sex,time,total_bill,tip,size
0,Male,Lunch,28.440000,2.560000,2.000000
1,Male,Dinner,18.616667,2.928333,2.666667
2,Female,Lunch,12.740000,2.260000,2.000000
3,Female,Dinner,15.380000,3.000000,2.000000


* 그룹 오브젝트 활용 참고
* https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html